In [ ]:
# importing the cv2 library
import cv2

# loading the Haar Cascade algorithm file into alg variable
alg = "haarcascade_frontalface_default.xml"

# passing the algorithm to OpenCV
haar_cascade = cv2.CascadeClassifier(alg)

# loading the image path into file_name variable
file_name = '<INSERT YOUR IMAGE NAME HERE> for eg-> X1.jpg'

# reading the image
img = cv2.imread(file_name, 0)

# creating a black and white version of the image
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

# detecting the faces
faces = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.05, minNeighbors=2, minSize=(100, 100))

# for each face detected
for x, y, w, h in faces:
    # crop the image to select only the face
    cropped_image = img[y : y + h, x : x + w]

    # loading the target image path into target_file_name variable
    target_file_name = '<INSERT YOUR OUTPUT FACE IMAGE NAME HERE> for eg-> X2.jpg'
    cv2.imwrite(target_file_name, cropped_image)


In [ ]:
# importing the required libraries
import numpy as np
from imgbeddings import imgbeddings
from PIL import Image

# loading the face image path into file_name variable
file_name = '<INSERT YOUR FACE FILE NAME> (X2.jpg)'

# opening the image
img = Image.open(file_name)

# loading the `imgbeddings`
ibed = imgbeddings()

# calculating the embeddings
embedding = ibed.to_embeddings(img)[0]


In [ ]:
!psql <SERVICE_URI>
!CREATE EXTENSION vector;
!CREATE TABLE pictures (picture text PRIMARY KEY, embedding vector(768));


In [ ]:
# importing the required libraries
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect('<SERVICE_URI>')
cur = conn.cursor()
cur.execute('INSERT INTO pictures values (%s,%s)', (file_name, embedding.tolist()))
conn.commit()
conn.close()


In [ ]:
# loading the image path into file_name variable
file_name = '<INSERT THE FILE NAME FOR WHICH YOU WANT TO CHECK THE SIMILARITY>'

# reading the image
img = cv2.imread(file_name, 0)

# creating a black and white version of the image
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

# detecting the faces
faces = haar_cascade.detectMultiScale(gray_img, scaleFactor=1.05, minNeighbors=2, minSize=(100, 100))
# for each face detected in the image
for x, y, w, h in faces:
    # crop the image to select only the face
    cropped_image = img[y : y + h, x : x + w]

    # Convert the NumPy array to a PIL image
    pil_image = Image.fromarray(cropped_image)

    ibed = imgbeddings()

    # calculating the embeddings
    slack_img_embedding = ibed.to_embeddings(pil_image)[0]


In [ ]:
conn = psycopg2.connect()
cur = conn.cursor()
string_rep = "[" + ",".join(str(x) for x in slack_img_embedding.tolist()) + "]"
cur.execute("SELECT picture FROM pictures ORDER BY embedding <-> %s LIMIT 5;", (string_rep,))
rows = cur.fetchall()
for row in rows:
    print(row)
